# Exploring Hacker News Posts

Hacker News is a website popular among the tech community and is similar to Reddit in that users can submit posts and others can either upvote or downvote those posts. Simialar to Reddit, posts with most upvotes end up on the front page of Hacker News.

- Our main objective in this project is to see whether posts starting with Ask HN (posts that ask the Hacker News community a question) or Show HN (post that show a project, products, etc) receive more comments on average?

- Another aim of this project is to try to determine whether posting at a certain time will result in more comments on average?

In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
list_file = list(read_file)
hn = list_file[1:]
headers = list_file[0]

print(headers)
print('\n')
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


To answer our first question, we need to separate posts that start with either Ask HN or Show HN.

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


Our next step is to determine the average number of comments for both Ask HN and Show HN posts.

In [3]:
total_ask_comments = 0
total_show_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
avg_ask_comments = round(total_ask_comments / len(ask_posts), 2)
avg_show_comments = round(total_show_comments / len(show_posts), 2)

print ('Ask HN posts have an average of {} comments per post.'.format(avg_ask_comments))
print ('Show HN posts have an average of {} comments per post.'.format(avg_show_comments))

Ask HN posts have an average of 14.04 comments per post.
Show HN posts have an average of 10.32 comments per post.


On average, Ask HN posts receive almost 4 comments per post than Show HN posts.
Because of this, our further analysis will focus on Ask HN posts. We'll try to determine whether the time of posting affects the number of comments. To do so, we first need to import the datetime module.

In [4]:
import datetime as dt

result_list = []

for post in ask_posts:
    post_time = post[6]
    num_comments = int(post[4])
    result_list.append([post_time, num_comments])
    
counts_by_hour = {}
comments_by_hour = {}

for result in result_list:
    time = dt.datetime.strptime(result[0], '%m/%d/%Y %H:%M')    
    hour = time.strftime('%H')
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = result[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += result[1]
        
print('Counts by hour:', counts_by_hour)
print('\n')
print('Comments by hour:', comments_by_hour)

Counts by hour: {'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


Comments by hour: {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


We'll next calculate the average number of comments for posts created during each hour of the dat.

In [5]:
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([round(comments_by_hour[hour] / 
                                  counts_by_hour[hour], 2), hour])

print(avg_by_hour)

[[5.58, '09'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [16.8, '16'], [7.99, '23'], [9.41, '12'], [11.46, '17'], [38.59, '15'], [16.01, '21'], [21.52, '20'], [23.81, '02'], [13.2, '18'], [7.8, '03'], [10.09, '05'], [10.8, '19'], [11.38, '01'], [6.75, '22'], [10.25, '08'], [7.17, '04'], [8.13, '00'], [9.02, '06'], [7.85, '07'], [11.05, '11']]


We'll finish by sorting the list of lists (avg_by_hour) and displaying the five highest values in a format that's easier to read.

In [6]:
sorted_avg = sorted(avg_by_hour, reverse=True)

print('Top 5 Hours for Ask Posts Comments \n')

for row in sorted_avg[:5]:
    hour = dt.datetime.strptime(row[1], '%H')
    hour = hour.strftime('%H:%M')
    avg_comments = row[0]
    print('{}: {} average comments per post'.format(hour, avg_comments))

Top 5 Hours for Ask Posts Comments 

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.8 average comments per post
21:00: 16.01 average comments per post


To receive more comments on your Ask HN posts, consider posting 
- In the afternoon: 15:00 and 16:00 received 38.59 and 21.52 comments respectively. 
- Evening times, 20:00 and 21:00 seem to also be doing well in terms of comment, each receiving on average 21.52 and 16.01 comments. 
- Finally, posting at 2:00 at night will result in 23.81 comments on average.

---

Let's now tackle some additional questions:

- Do ask or show posts receive more points on average?
- Do posts created at a certain time receive more points?

These two questions are very similar in nature to the ones already analyzed in this project, so we'll dive right in trying to answer them. The column of interest is num_points, with an index of 3.

In [7]:
total_ask_points = 0
total_show_points = 0

for post in ask_posts:
    num_points = int(post[3])
    total_ask_points += num_points
    
for post in show_posts:
    num_points = int(post[3])
    total_show_points += num_points
    
avg_ask_points = round(total_ask_points / len(ask_posts), 2)
avg_show_points = round(total_show_points / len(show_posts), 2)

print ('Ask HN posts have an average of {} points per post.'.format(avg_ask_points))
print ('Show HN posts have an average of {} points per post.'.format(avg_show_points))

Ask HN posts have an average of 15.06 points per post.
Show HN posts have an average of 27.56 points per post.


We can see that Show HN posts on average receive 12.5 points more than Ask HN posts. We can speculate that this has to do with the nature of the post, in which a member shares somethis he/she has done, such as a project, website, etc. and people upvote it more to show their appreciation. In the next question, whether certain posting at certain times can result in more upvotes, we'll focus on Show HN question types, because of the results of average points per post.

In [8]:
show_hn_upvotes = {}

for post in show_posts:
    points = int(post[3])
    date = post[-1]
    time = dt.datetime.strptime(date, '%m/%d/%Y %H:%M')
    hour = time.strftime('%H')
    
    if hour not in show_hn_upvotes:
        show_hn_upvotes[hour] = points
    else:
        show_hn_upvotes[hour] += points
        
print(show_hn_upvotes)

{'14': 2187, '22': 1856, '18': 2215, '07': 494, '20': 1819, '05': 104, '16': 2634, '19': 1702, '15': 2228, '03': 679, '17': 2521, '06': 375, '02': 340, '13': 2438, '08': 519, '21': 866, '04': 386, '11': 1480, '12': 2543, '23': 1526, '09': 553, '01': 700, '10': 681, '00': 1173}


In [9]:
avg_points_by_hour = []

for hour in counts_by_hour:
    hour = hour
    points = show_hn_upvotes[hour]
    avg_points_by_hour.append([round(points / counts_by_hour[hour],2), hour])
    
avg_points_by_hour = sorted(avg_points_by_hour, reverse=True)
print(avg_points_by_hour[:5])

[[34.84, '12'], [28.68, '13'], [26.14, '22'], [25.52, '11'], [25.21, '17']]


On average, Show HN posts in the first half of the day, 11am, 12pm and 1pm seem to be getting more upvotes, with 34.84, 28.68 and 25.52 points respectively. Show HN posts later in the day, namely at 5pm and 10pm, get 25.21 and 26.14 points.